In [1]:
!pip install faker

In [238]:
import numpy as np
import pandas as pd
import csv
from faker import Faker
import random

In [239]:
fake = Faker()

In [258]:
rand_factor = 1/100       # 1/100 will include a maximum noice of 1000 Mts (basically rand_factor*1L)
timestep = 1
man_steps = 5
m = 20
t = 3600
# n = 3600

speed = 220 / 100000
avg_alt = 5000 / 100000
alt_var = 250 / 100000
take_off = False
maneuver = 0 #To be given a value between 0 and 1 like a boolean and intermediate values will define that much degree of maneuver
fighter = True
maneuver_fact = maneuver * 2 / 1000
maneuver_prob = 0 #set this to be the number of times your plane has to maneuver per 1 move setting this to zero will make no maneuver and 1 will make random motion towards the target dest
regular_time = True

In [259]:
src_lat, src_lon , d_lat , d_lon = float(fake.latitude()), float(fake.longitude()), float(fake.latitude()), float(fake.longitude())

In [260]:
max_dist = 1000000 / 100000 
d_lat = src_lat + ((np.random.normal(0,1,1)+0.5)*max_dist)
d_lon = src_lon + (np.random.normal(0,1,1)+0.5)*max_dist

d_lat, d_lon = d_lat[0],d_lon[0]
print(d_lat,d_lon)

67.90810493353457 152.2011734839757


In [261]:
#Defining the path of latitude

pts_lat=[src_lat]
for i in range(m):
    new_src_lat = src_lat+(d_lat-src_lat+random.random()*rand_factor)/(m-i+1)
    pts_lat.append(new_src_lat)
    src_lat = new_src_lat
pts_lat.append(d_lat)
print(pts_lat)


#Defining the path of longitude

pts_lon=[src_lon]
for i in range(m):
    new_src_lon = src_lon+(d_lon-src_lon+random.random()*rand_factor)/(m-i+1)
    pts_lon.append(new_src_lon)
    src_lon = new_src_lon
pts_lon.append(d_lon)

print(pts_lon)

[59.2577215, 59.670075543796365, 60.08222991417507, 60.494609990396945, 60.90684742342294, 61.31889938401002, 61.731118780066126, 62.14303663240617, 62.55540523461645, 62.96738612438311, 63.37986333241847, 63.79188130328861, 64.20352389315342, 64.61576522519688, 65.02828008567916, 65.44008479296357, 65.85286641615228, 66.26494380636467, 66.67784230267154, 67.08994982429654, 67.49977969724719, 67.90810493353457]
[142.060086, 142.54321809316093, 143.02611980416282, 143.50953605477864, 143.9927695591033, 144.4758423400358, 144.95924590353957, 145.44221144737173, 145.9254977486906, 146.40863036954667, 146.89158414092105, 147.37497370536104, 147.8582038677933, 148.34183818287914, 148.82542137990353, 149.30894139483328, 149.79173514887165, 150.27401865919657, 150.75744593549177, 151.2408282563091, 151.72125114001085, 152.2011734839757]


In [262]:
def set_factor(pos,dest_pos,vel,acc,max_vel):
  next_pt = update_pos(pos,vel,acc,0)
  fact = [0,0,0]
  if(np.linalg.norm(vel+acc)>max_vel):
    # print("parithi")
    return [0,0,0]
  target = np.subtract(dest_pos,next_pt)
  # diff = (np.multiply(target,acc)).tolist()
  # diff = [diff[0]>0,diff[1]>0,diff[2]>0]
  dir_target = np.divide(target,np.add(np.absolute(target),0.000000000001))
  dir_vel = np.divide(target,np.add(np.absolute(vel),0.000000000001))
  diff = (dir_target == dir_vel)
  diff = [int(i == False) for i in diff]
  fact = np.multiply(dir_target,np.divide(np.absolute(target),np.add(magnitude(dir_target),0.000000000001))/100)
  # print((np.multiply(fact,diff)))
  return (np.multiply(fact,diff)).tolist()

def magnitude(pos):
  return np.linalg.norm(pos)

def dist(pos1,pos2):
  return np.sqrt(np.sum(np.square(np.subtract(pos1,pos2))))

def air_resist(acc):
  return (np.divide(acc,10)).tolist()

def update_pos(pos,vel,acc,noise):
  if(magnitude(acc)>0.00000000000001):
    v = (np.add(vel , acc)).tolist()
    s = (np.divide(np.subtract(np.square(v),np.square(vel)),np.multiply(acc,2))).tolist()
    # print((np.add(np.add(pos,s),np.multiply(np.random.normal(0,1,3),noise))).tolist())
    return (np.add(np.add(pos,s),np.multiply(np.random.normal(0,1,3),noise))).tolist()
  return (np.add(pos,vel)).tolist()

def update_vel(vel,acc,noise):
  return (np.add(np.add(vel,acc),np.random.normal(0,1,3)*noise)).tolist()

def update_acc(pos,dest_pos,vel,max_vel,acc,noise):
  new_acc = np.add(acc,np.random.normal(0,1,3)*noise)
  new_acc = np.add(new_acc,np.multiply(np.random.rand(3),set_factor(pos,dest_pos,vel,acc,max_vel)))
  new_acc = np.subtract(new_acc,air_resist(new_acc.tolist()))
  return new_acc.tolist()

def has_crossed(pos,dest_pos,vel,max_vel,acc,noise):
  pres_dir=np.divide(np.subtract(pos,dest_pos),np.absolute(np.subtract(pos,dest_pos)))
  next_pos=update_pos(pos,vel,acc,noise)
  next_dir=np.divide(np.subtract(next_pos,dest_pos),np.absolute(np.subtract(next_pos,dest_pos)))
  return (pres_dir!=next_dir)

  # To be completed before i finish dinner
  pass

def intrapol(init_pos,vel,acc,speed,maneuver_prob,maneuver_fact,noise_fact):
  global timestep, man_steps
  max_vel = speed
  [src_lat,src_lon,src_alt,dest_lat,dest_lon,dest_alt] = init_pos
  dest_pos = [dest_lat,dest_lon,dest_alt]
  [x_vel,y_vel,z_vel] = vel
  [x_acc,y_acc,z_acc] = acc
  lats , lons , alts = [src_lat] , [src_lon] , [src_alt]
  pos = [src_lat,src_lon,src_alt]
  man = False
  while( not (has_crossed(pos,dest_pos,vel,max_vel,acc,0)).any()):
#   for i in range(100):
    if(man == True):
      pass
    else:
      # pos , vel , acc = update_pos(pos,vel,acc,noise_fact[0]), update_vel(vel,acc,noise_fact[1], update_acc(acc,noise_fact[2]))
      pos = update_pos(pos,vel,acc,noise_fact[0])
      vel = update_vel(vel,acc,noise_fact[1])
      acc = update_acc(pos,dest_pos,vel,max_vel,acc,noise_fact[2])
      lats.append(pos[0])
      lons.append(pos[1])
      alts.append(pos[2])
      print (pos , ( not ((has_crossed(pos,dest_pos,vel,max_vel,acc,noise_fact[0])).any())))
      # man = (random.random() < maneuver_prob)
  return {"pos":[lats, lons, alts],"vel":[x_vel,y_vel,z_vel],"acc":[x_acc,y_acc,z_acc],"last_pt":[lats[-1],lons[-1],alts[-1]]}

# print(update_pos([0,0,0],[4,6.45,4],[12,3,2],2))
# print(update_vel([0,0,0],[12,3,2],2))
# print(update_acc([12,3,2],2))
# print(dist([1,1,1],[0,0,0]))
# print(np.linalg.norm([1,1,1]))

In [263]:
orig_lat = []
orig_lon = []
orig_alt = []
alt_src = avg_alt + (random.random()-0.5)*2*alt_var
if(take_off == False):
    vel = [random.random()*speed,random.random()*speed,0]
    # vel[1] = np.sqrt(speed - np.square(vel[0]))
    acc = [0,0,0]
    print(np.array(acc).shape)
    for i in range(len(pts_lat)-1):
    # for i in range(2):
        alt_dst = avg_alt + (random.random()-0.5)*2*alt_var
        pos = [pts_lat[i],pts_lon[i],alt_src,pts_lat[i+1],pts_lon[i+1],alt_dst]
        print("========================"+str((pts_lat[i],pts_lon[i]))+" "+str(alt_src)+"=========================================")
        res = intrapol(pos,vel,acc,speed,maneuver_prob,maneuver_fact,[0,0,0])
        vel = res["vel"]
        acc = res["acc"]
        vect = res["pos"]
        # print(res["pos"])
        alt_src = alt_dst
        [pts_lat[i+1],pts_lon[i+1],alt_src] = res["last_pt"]
        orig_lat.extend(vect[0])
        orig_lon.extend(vect[1])
        orig_alt.extend(vect[2])
        
else:
    for i in range(len(pts_lat)-3):
        pass


# print(np.array([orig_lat,orig_lon,orig_alt]))
print(len(orig_lat))

(3,)
========================(59.2577215, 142.060086) 0.048723561185564694=========================================
[59.25892088981058, 142.06053771565013, 0.048723561185564694] True
[59.26113712536975, 142.0615945790777, 0.04873043937555505] True
[59.26528536785124, 142.06380122328244, 0.04875038612652709] True
[59.27117241656282, 142.0670426701866, 0.048782094582382626] True
[59.278624390881475, 142.07121543952024, 0.048824388572633326] True
[59.2874847982465, 142.07622639904042, 0.04887620954383967] True
[59.29761279535327, 142.0819917297285, 0.04893660479790609] True
[59.30888162322759, 142.08843599446766, 0.049004716906546585] True
[59.321177198792725, 142.09549129985282, 0.04907977418430374] True
[59.334396847279585, 142.10309654181938, 0.04916108211426589] True
[59.34844816139599, 142.11119672670918, 0.04924801563121254] True
[59.363247974579004, 142.1197423602299, 0.049340012176445236] True
[59.37872143692195, 142.12868889751846, 0.04943656544713538] True
[59.39480118350884, 14

[63.771372483206925, 147.22164084622776, 0.04905590309496651] False
========================(63.771372483206925, 147.22164084622776) 0.04905590309496651=========================================
[63.772571873017505, 147.22209256187787, 0.04905590309496651] True
[63.77414082848559, 147.22338946240666, 0.04905966263882131] True
[63.77641195870295, 147.22629221420493, 0.049070565316000236] True
[63.77931504619466, 147.2306402321457, 0.04908789681317088] True
[63.78278689523328, 147.23628898961476, 0.049111014248334056] True
[63.78677062966412, 147.24310841265927, 0.04913933902769052] True
[63.791215060947955, 147.25098143472167, 0.04917235041682095] True
[63.79607411939949, 147.25980269590016, 0.049209579754747934] True
[63.801306342301956, 147.26947737228318, 0.049250605246591826] True
[63.80687441321025, 147.27992012235023, 0.049295047276960935] True
[63.812744747323805, 147.29105413873293, 0.04934256419200274] True
[63.81888711832208, 147.3028102947997, 0.049392848503249966] True
[63.82

In [264]:
data = np.array([range(len(orig_alt)),orig_lat,orig_lon,orig_alt])
data = np.transpose(data)
data = np.multiply(data,[1,1,1,100000])
d = pd.DataFrame(data=data,columns = ["t","lat","lon","alt"])
d.to_csv('traj'+str(count)+'.csv', sep=",", index=False)

count= count + 1

In [265]:
print(count)

61


In [266]:
count = 60